In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
import gensim

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve, precision_score, recall_score
from sklearn.metrics import classification_report

from sklearn.metrics import classification_report, f1_score


#text is already cleaned.
#assign cleaned data to these variables.
train_data_path = 'cleaned_data/cleaned_train_data_for_subtask1.csv'
test_data_path = 'cleaned_data/cleaned_test_data_for_subtask1.csv'
#read files.
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

print("Train set:"% train_data.columns, train_data.shape, len(train_data)) 
print("Test set:"% test_data.columns, test_data.shape, len(test_data)) 



C:\Users\Mena\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Train set: (20974, 8) 20974
Test set: (4997, 8) 4997


In [2]:
#prepare train and test data.
X_train = train_data['#2_tweet_clean_V3']
y_train = train_data['#classes_id']
X_test = test_data['#2_tweet_clean_V3']
y_test = test_data['#classes_id']

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

max_no = 50000
sql_len = 250
dim = 100

tokenizer = Tokenizer(num_words=max_no, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(X_train.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

tokenizer = Tokenizer(num_words=max_no, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=False)
tokenizer.fit_on_texts(X_test.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 48685 unique tokens.
Found 17156 unique tokens.


In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


X_tr = tokenizer.texts_to_sequences(X_train.values)
X_tr = pad_sequences(X_tr, maxlen=sql_len)
print('Shape of data tensor:', X_tr.shape)

X_te = tokenizer.texts_to_sequences(X_test.values)
X_te = pad_sequences(X_te, maxlen=sql_len)
print('Shape of data tensor:', X_te.shape)


Shape of data tensor: (20974, 250)
Shape of data tensor: (4997, 250)


In [5]:
Y_tr = pd.get_dummies(y_train).values
print('Shape of label tensor:', Y_tr.shape)

Y_te = pd.get_dummies(y_test).values
print('Shape of label tensor:', Y_te.shape)

Shape of label tensor: (20974, 21)
Shape of label tensor: (4997, 21)


In [7]:
import keras
import tensorflow
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
#import tensorsflow_addons as tfa
model = keras.Sequential()
model.add(Embedding(max_no, dim, input_shape=(sql_len,)))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 20
batch_size = 64

history = model.fit(X_tr, Y_tr, epochs=epochs, batch_size=batch_size,validation_split=0.3)

Train on 14681 samples, validate on 6293 samples
Epoch 1/20
14681/14681 [==============================] - 261s 18ms/step - loss: 2.6343 - accuracy: 0.2097 - val_loss: 2.5464 - val_accuracy: 0.2547
Epoch 2/20
14681/14681 [==============================] - 320s 22ms/step - loss: 2.3768 - accuracy: 0.3118 - val_loss: 2.3889 - val_accuracy: 0.3013
Epoch 3/20
14681/14681 [==============================] - 387s 26ms/step - loss: 2.0611 - accuracy: 0.4118 - val_loss: 2.3498 - val_accuracy: 0.3240
Epoch 4/20
14681/14681 [==============================] - 369s 25ms/step - loss: 1.7577 - accuracy: 0.4964 - val_loss: 2.4299 - val_accuracy: 0.3273
Epoch 5/20
14681/14681 [==============================] - 364s 25ms/step - loss: 1.5093 - accuracy: 0.5651 - val_loss: 2.5680 - val_accuracy: 0.3248
Epoch 6/20
14681/14681 [==============================] - 360s 25ms/step - loss: 1.4074 - accuracy: 0.6034 - val_loss: 2.6816 - val_accuracy: 0.3189
Epoch 7/20
14681/14681 [==============================] -

In [9]:
y_pred= model.predict(X_te)
y_pred = np.argmax(y_pred, axis=1)
y_test = test_data['#classes_id']
Y_te = pd.get_dummies(y_test).values

Y_te = np.argmax(Y_te, axis=1)
print(f1_score(Y_te, y_pred, average='macro'))


0.14237240059889103


In [10]:
classes = list(train_data['#3_country_label'].unique())
print(classification_report(Y_te, y_pred,target_names=classes))

                      precision    recall  f1-score   support

               Egypt       0.53      0.55      0.54      1041
                Iraq       0.36      0.39      0.38       663
        Saudi_Arabia       0.20      0.21      0.21       519
          Mauritania       0.19      0.25      0.21        53
             Algeria       0.32      0.36      0.34       430
               Syria       0.11      0.15      0.13       278
                Oman       0.13      0.13      0.13       355
             Tunisia       0.13      0.15      0.14       172
             Lebanon       0.07      0.05      0.06       157
             Morocco       0.09      0.09      0.09       207
            Djibouti       0.00      0.00      0.00        27
United_Arab_Emirates       0.11      0.08      0.09       157
              Kuwait       0.07      0.08      0.07       105
               Libya       0.24      0.19      0.21       314
             Bahrain       0.00      0.00      0.00        52
       